In [2]:
from rusyll.rusyll import token_to_syllables
from nltk import word_tokenize, sent_tokenize 
from nltk.tokenize import SyllableTokenizer
import numpy as np
import warnings

In [4]:
text = """Будучи ребёнком, Кусуноки думал, что его ждёт великое будущее. Однако прошло десять лет и наивные детские мечты сменились суровой реальностью. Жизнь Кусуноки сера и безрадостна. Двадцатилетний парень работает в кафе, и денег ему не хватает даже на еду. Распродавая свои книги и диски, он узнаёт от знакомого букиниста об особом магазине, в котором скупают жизни людей. При этом стоимость меняется в зависимости от человека. Восприняв это как несмешную шутку, парень решает всё же сам всё проверить. Так как за свою жизнь японец в среднем тратит около 200—300 миллионов иен, примерно на такую сумму Кусуноки и рассчитывает. Однако оставшуюся Кусуноки жизнь нельзя назвать счастливой и в ней не будет никаких достижений, поэтому, основываясь на наличии таких факторов, как счастье, актуализация и вклад, он обнаруживает, что за каждый год магазин готов предложить лишь минимальную цену в десять тысяч иен. Хотя Кусуноки и рассчитывал долго прожить, в настоящем исходе ему остаётся не более тридцати лет. В итоге Кусуноки решает воспользоваться этой возможностью и продаёт тридцать лет своей жизни, оставив себе только три месяца. На оставшийся срок к нему приставляют наблюдательницу Мияги, которая должна гарантировать, что её подопечный не будет создавать проблем другим людям. """

In [20]:
def get_flash_readibility(text, lang, type):
    if lang not in ["english", "russian"]:
        raise NotImplementedError(lang)
    tokens = word_tokenize(text, lang)
    sents = sent_tokenize(text, lang)
    mean_sent_len = np.mean([len(x.split(' ')) for x in sents])
    if lang == "russian":
        mean_word_len = np.mean([len(token_to_syllables(x)) for x in tokens])
        if type == "flash":
            readibility = 206.836-60.1*mean_word_len-1.3*mean_sent_len
        elif type == "flash-kinside":
            readibility = 0.5*mean_sent_len + 8.4*mean_word_len - 15.59

    else:
        SSP = SyllableTokenizer()
        mean_word_len = np.mean([len(SSP.tokenize(x)) for x in tokens])
        if type == "flash":
            readibility = 206.836-84.6*mean_word_len-1.015*mean_sent_len
        elif type == "flash-kinside":
            readibility = 0.39*mean_sent_len + 11.8*mean_word_len - 15.59
    return readibility

def fog_index(text, lang):
    tokens = word_tokenize(text, lang)
    if len(tokens) < 100:
        warnings.warn(f"Text should contain at least 100 tokens. You have {len(tokens)}")
    middle = len(tokens) // 2
    selected_tokens = tokens[middle-50:middle] + tokens[middle:middle+50]
    sents = sent_tokenize(' '.join(selected_tokens), lang)
    mean_sent_len = np.mean([len(x.split(' ')) for x in sents])
    if lang == "russian":
        word_syll = [len(token_to_syllables(x)) for x in tokens]
    else:
        SSP = SyllableTokenizer()
        word_syll = [len(SSP.tokenize(x)) for x in tokens]
    complex_words = list(filter(lambda x: x > 2, word_syll))
    complex_words_frac = len(complex_words) / len(selected_tokens)
    if lang == "english":
        fog_index = (mean_sent_len + complex_words_frac) * 0.4
    else:
        fog_index = (1.25*mean_sent_len + 0.24*complex_words_frac) * 0.4
    return fog_index


In [21]:
fog_index(text, "russian")

10.073920000000001

2.2146118721461185